<a href="https://colab.research.google.com/github/gt-cse-6040/bootcamp/blob/main/Module%201/Self%20Assessments/Module%201%20Self%20Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self-Assessment for Module 1
Module 1 in this course focuses on your ability to manipulate and analyze data using Pandas, SQL, and Numpy. Use this assessment to test your existing knowledge. You should be able to complete all five exercises within 90 minutes.

If you are unable to complete the exercises within 90 minutes, strongly consider attending the Module 1 Bootcamp sessions we offer. They will build the skills necessary to succeed within the second half of the course.

When you are ready, start a [countdown timer](https://www.youtube.com/watch?v=IG9o_I96Y78). Let's get to coding!


# Overview: Exam Analysis #

An instructor for another course has asked you to help analyze some of their exam data. The exams were completed electronically, in a format somewhat similar to our class's:

- The class has multiple exams, named `'exam1'`, `'exam2'`, and so on. Exams were timed, but with different time limits for each exam.
- Each exam consisted of multiple "milestones" (or "problems"). There were different numbers of problems for each exam, and different problems can be worth different numbers of points. (Thus, the total points on different exams could also differ.)
- Students were allowed to work on the problems in any order and submit in any order, similar to our exams.

The dataset itself has **two** main parts:
- The **grades data**, which shows, for each student, exam, and problem, how many points they scored.
- The **timing data**, which shows, for each student and problem, when (i.e., at what date and time) the student started the problem and made their last submission thereof.


# Setup #

To get started, run the following code cells:

In [1]:
!pip install dill
import dill
!pip install cryptography

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.1 MB/s eta 0:00:00


In [2]:
### Import Files

!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/tc_2
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/tc_5
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/tc_4

!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/encrypted/tc_2_encrypted
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/encrypted/tc_5_encrypted
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/encrypted/tc_4_encrypted

!mkdir tester_fw
%cd tester_fw

!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/tester_fw/__init__.py
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/tester_fw/test_utils.py
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/tester_fw/testers.py

%cd ..


--2024-01-08 02:00:05--  https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/tc_2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246776 (241K) [text/plain]
Saving to: ‘tc_2’

tc_2                100%[===================>] 240.99K  --.-KB/s    in 0.03s   

2024-01-08 02:00:06 (8.78 MB/s) - ‘tc_2’ saved [246776/246776]

--2024-01-08 02:00:06--  https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/tc_5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP reque

In [3]:
### Global Imports
%load_ext autoreload
%autoreload 2

# Modules you may find useful:
import pandas as pd
import numpy as np

# Some extra functions that this notebook needs:
from pprint import pprint # Pretty-printer

import time
overall_start = time.time()

def dfs_to_conn(conn_dfs, index=False):
    import sqlite3
    conn = sqlite3.connect(':memory:')
    for table_name, df in conn_dfs.items():
        df.to_sql(table_name, conn, if_exists='replace', index=index)
    return conn

# Load the dataset:
print("\n==> Loading the dataset...")
grades_exam1 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/grades_exam1.csv")
grades_exam2 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/grades_exam2.csv")
grades_exam3 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/grades_exam3.csv")

times_exam1 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/timings_exam1.csv")
times_exam2 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/timings_exam2.csv")
times_exam3 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/timings_exam3.csv")

grades_by_exid = {'exam1': grades_exam1, 'exam2': grades_exam2, 'exam3': grades_exam3}
times_by_exid = {'exam1': times_exam1, 'exam2': times_exam2, 'exam3': times_exam3}

print("\n==> Done executing the setup and ready for take-off!")


==> Loading the dataset...

==> Done executing the setup and ready for take-off!


# Exercise 0: `calc_earned_points` #

Suppose you have a grades dataframe, `valid_gdf`. Complete the function,
```python
def calc_earned_points(valid_gdf):
    ...
```
so that it returns a new copy of `valid_gdf` with an additional column holding the total points earned.

**Input:** A dataframe called `valid_gdf` containing the grades by student for each problem on an exam.

**Your task:** Create a copy and sum the points earned by each student.

**Output:** Return a new dataframe with the same columns as `valid_gdf` _plus_ an additional column, `'Earned'`, of `.dtype` `float`. This column should contain the student's total score, that is, the sum of points on their problems. Any NaN values should be ignored. You should not modify the input `valid_gdf`.

## Exercise 0 demo input ##

In [ ]:
### Define demo inputs ###
grades_by_exid_1 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/valid_grades_exam1.csv")
demo_gdf_ex0 = grades_by_exid_1.sample(6, random_state=3_456_789_012).reset_index(drop=True)
demo_gdf_ex0

The demo included in the solution cell below should display the following output:

|    | Email                    |   problem0 |   problem1 |   problem2 |   problem3 |   problem4 |   Earned |
|---:|:-------------------------|-----------:|-----------:|-----------:|-----------:|-----------:|---------:|
|  0 | rene.amritpal@gmail.com  |         10 |         10 |       10   |         10 |         10 |     50   |
|  1 | albert.averley@gmail.com |         10 |          2 |        8   |         10 |         10 |     40   |
|  2 | mary.yahshua@gatech.edu  |          7 |          2 |        6.5 |          5 |        nan |     20.5 |
|  3 | jillian.myler@gmail.com  |         10 |         10 |       10   |         10 |          1 |     41   |
|  4 | raina.sheehan@gmail.com  |         10 |         10 |       10   |          5 |         10 |     45   |
|  5 | allison.link@yahoo.com   |          7 |          2 |      nan   |        nan |        nan |      9   |

> **Note:** Although the values appear to include integers, in fact, the `.dtype` of the columns should be `float`.

## Exercise 0 your code ##

In [ ]:
### Exercise 0 solution ###
def calc_earned_points(valid_gdf):
    ### BEGIN SOLUTION

    ### END SOLUTION

### demo function call ###
calc_earned_points(demo_gdf_ex0)

<!-- Test Cell Boilerplate -->
## Exercise 0 test ##
The cell below will test your solution for Exercise 0. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex0 ###
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_2',
    'func': calc_earned_points, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'valid_gdf':{
            'dtype':'df', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}

conf_encrypted = {
    'case_file':'tc_2_encrypted',
    'func': calc_earned_points, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'valid_gdf':{
            'dtype':'df', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'It2jBzT52UbvyXCvc19bZBXTaLtU-OurcEtWwuHet8M=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### BEGIN HIDDEN TESTS
tester = Tester(conf_encrypted, key=b'dRaQLzH9Dx27yR5MBt5Iy3HBuRaSpg1bXvN8NFqZ8j4=', path='')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
### END HIDDEN TESTS
print('Passed! Please continue.')

## Exercise 0 RUNME (precomputed solutions) ##

The cell below creates a precomputed object, `earned_by_exid`. It is the result of a correct implementation of `calc_earned_grades` when run on the full dataset. Subsequent cells use it, so please run this cell regardless of whether you completed this exercise.

In [ ]:
# Precomputed solutions
earned_by_exid_1 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/earned_grades_exam1.csv")
earned_by_exid_2 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/earned_grades_exam2.csv")
earned_by_exid_3 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/earned_grades_exam3.csv")
earned_by_exid = {'exam1': earned_by_exid_1, 'exam2': earned_by_exid_2, 'exam3': earned_by_exid_3}

# Example:
display(earned_by_exid_1.iloc[:4])
earned_by_exid_1.info()

Similar to the grades dataset, each exam has a **timings** dataset.

**Observations:**
- Each row is for a student and a problem ("milestone" column). There may be multiple rows for the same student, but a (student, problem) pair appears _at most_ once.
- The `'start date-time'` and `'submit date-time'` columns hold timestamps, and their `.dtypes` are special pandas [`Timestamp`](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp) objects.
- The `'start date-time'` value is when the student began the exam. It will have the same value for all rows of a given student.
- The `'submit date-time'` value is the timestamp of the student's _last_ submission for that problem. That is, only one submission is recorded per student and problem.

Here's what it looks like for `'exam3'`:

In [ ]:
demo_times_exam3 = times_by_exid['exam3']
demo_times_exam3

# Exercise 1: `clean_times_colnames` #

Let's tidy up the column names for the timings dataframe. Complete the function,
```python
def clean_times_colnames(tdf):
    ...
```
to return a new dataframe from `tdf` with columns renamed as follows.

* `'email'` => `'Email'`
* `'milestone'` => `'Milestone'`
* `'start date-time'` => `'Start'`
* `'submit date-time'` => `'Submit'`

Element types for the columns (`.dtype`) should be preserved, and the input dataframe should not be modified.

## Exercise 1 demo input ##

In [ ]:
### define demo inputs ###
valid_times_by_exid_2 = pd.read_csv("https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Self%20Assessments/resource/asnlib/publicdata/valid_times_exam2.csv")
demo_tdf_ex1 = valid_times_by_exid_2.sample(5, random_state=567_890_123, replace=False).reset_index(drop=True)
demo_tdf_ex1

The demo included in the solution cell below should display the following output:

|    | Email                       | Milestone   | Start               | Submit              |
|---:|:----------------------------|:------------|:--------------------|:--------------------|
|  0 | jazlyn.florin@gmail.com     | problem4    | 2018-11-04 12:45:10 | 2018-11-05 20:50:41 |
|  1 | christian.miyisha@yahoo.com | problem2    | 2018-11-03 07:38:00 | 2018-11-04 12:21:12 |
|  2 | taryn.raneshia@yahoo.co.id  | problem0    | 2018-11-04 08:25:24 | 2018-11-05 17:59:02 |
|  3 | joseph.sherburne@gmail.com  | problem1    | 2018-11-02 16:45:06 | 2018-11-03 01:47:58 |
|  4 | thomas.dyanni@gmail.com     | problem3    | 2018-11-03 14:20:56 | 2018-11-04 23:31:18 |

## Exercise 1 your code ##

In [ ]:
### Exercise 1 solution
def clean_times_colnames(tdf):
    ### BEGIN SOLUTION

    ### END SOLUTION

### demo function call ###
clean_times_colnames(demo_tdf_ex1)

## Exercise 1 test ##
The cell below will test your solution for Exercise 1. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex1
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_5',
    'func': clean_times_colnames, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'tdf':{
            'dtype':'df', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
conf_encrypted = {
    'case_file':'tc_5_encrypted',
    'func': clean_times_colnames, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'tdf':{
            'dtype':'df', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'It2jBzT52UbvyXCvc19bZBXTaLtU-OurcEtWwuHet8M=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### BEGIN HIDDEN TESTS
tester = Tester(conf_encrypted, key=b'dRaQLzH9Dx27yR5MBt5Iy3HBuRaSpg1bXvN8NFqZ8j4=', path='')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
### END HIDDEN TESTS
print('Passed! Please continue.')

**Timestamps:** The timestamps in the timings dataset have the following form:

```2018-12-07 18:35:23```

That corresponds to December 7, 2018, at 6:35 pm (and 23 seconds).

Luckily, you don't need to parse timestamps! These objects make it easy to sort and calculate time differences, which is what we'll need. For instance, we can check if one date is larger than another using the usual comparison-operators, like `>` or `<=`:

In [ ]:
(demo_times_exam3['submit date-time'] > demo_times_exam3['start date-time']).head()

**Some messiness:** The timings dataset has some "impurities." In particular, check out the data for `'kenneth.bryland@gatech.edu'`:

In [ ]:
demo_times_exam3_messy = demo_times_exam3.loc[[206, 1111, 1780, 2511, 3375]]
demo_times_exam3_messy

Observe that their `'problem0'` and `'problem1'` submission times occur **before** their start times. While it turns out there is a reason for that, we want to identify all instances where this problem occurs.

# Exercise 2: `remove_causality_violators` #

Suppose you are given a timings dataframe, `tdf`, per Exercise 1. Complete the function,
```python
def remove_causality_violators(tdf):
    ...
```
so that it returns a new, cleaned copy that omits records for any student who has timestamps that "violate causality." A causality violation means that their submission time is **strictly less** than their start time.

**Input:** A timings dataframe, `tdf`, with the columns `'email'`, `'milestone'`, `'start date-time'`, and `'submit date-time'`.

**Your task:** Identify all students who have any causality violations and omit their rows from the output dataframe.

**Output:** Return a new tibble that includes only rows from `tdf` for students whose problem-records do not violate causality. The output column `.dtypes` should match those of the input. You should not modify the input, `tdf`.

**Comment(s):** If a student has even one problem that violates causality, you should omit _all_ of their rows. That is, we will consider their entire exam data invalid for the purpose of our subsequent analysis.

**Hint:** The dataframe method [isin()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html) and the ~ operator may be useful for this exercise.

## Exercise 2 demo input ##

In [ ]:
### Define demo inputs ###
demo_tdf_ex2 = times_by_exid['exam3'].loc[[206, 279, 439, 1111, 1344, 1780, 2511, 3224, 3375]].reset_index(drop=True)
demo_tdf_ex2

The demo included in the solution cell below should display the following output:

|    | email                        | milestone   | start date-time     | submit date-time    |
|---:|:-----------------------------|:------------|:--------------------|:--------------------|
|  0 | daniel.tashera@gmail.com     | problem0    | 2018-12-09 01:12:56 | 2018-12-09 02:29:33 |
|  1 | barbara.wilhemina@gatech.edu | problem0    | 2018-12-10 02:43:28 | 2018-12-10 06:21:30 |
|  2 | barbara.wilhemina@gatech.edu | problem2    | 2018-12-10 02:43:28 | 2018-12-10 09:09:43 |
|  3 | barbara.wilhemina@gatech.edu | problem5    | 2018-12-10 02:43:28 | 2018-12-10 09:04:35 |

Observe that `kenneth.bryland@gatech.edu` is omitted from this output because their `'problem0'` and `'problem1'` records violate causality. (Even though their remaining problems _obey_ causality, we still drop them because of the problematic rows.)

## Exercise 2 your code ##

In [ ]:
### Exercise 2 solution ###
def remove_causality_violators(tdf):
    ### BEGIN SOLUTION

    ### END SOLUTION

### demo function call ###
remove_causality_violators(demo_tdf_ex2)

<!-- Test Cell Boilerplate -->
## Exercise 2 test ##
The cell below will test your solution for Exercise 2.

In [ ]:
### test_cell_ex2
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_4',
    'func': remove_causality_violators, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'tdf':{
            'dtype':'df', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
conf_encrypted = {
    'case_file':'tc_4_encrypted',
    'func': remove_causality_violators, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'tdf':{
            'dtype':'df', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'It2jBzT52UbvyXCvc19bZBXTaLtU-OurcEtWwuHet8M=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### BEGIN HIDDEN TESTS
tester = Tester(conf_encrypted, key=b'dRaQLzH9Dx27yR5MBt5Iy3HBuRaSpg1bXvN8NFqZ8j4=', path='')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
### END HIDDEN TESTS
print('Passed! Please continue.')

# Exercise 3: `compute_percentage_earned` #

Given a database connection `conn`, an email address `email`, a table name `exam_table`, and the maximum possible score `max_score`, complete the function,
```python
def compute_percentage_earned(conn, email, exam_table, max_score):
    ...
```
so that it returns a dataframe containing a student's exam percentage earned as described below.

**Inputs:**
- `conn`: A database connection.
- `email`: A string containing a student email address.
- `exam_table`: A string containing the name of the exam table to query. All exam tables contain the following columns: `'Email'`, `'Earned'`, and one or more `'problem..'` columns.
- `max_score`: A float containing the maximum number of points possible a student could achieve on the exam.

**Your tasks:**
Write a query that returns a dataframe with the following columns:
- `'Email'`: the student's email address
- `'Percentage Earned'`: The percentage the student earned on that exam, computed as `'Earned'` / `max_score` * 100.0

**Output:** Return the result as a dataframe with the columns `'Email'` and `'Percentage Earned'`.

**Hint:** [F-strings](https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/) may be useful for formatting your query in this exercise.

## Exercise 3 demo input ##

In [ ]:
### Define demo inputs ###
conn = dfs_to_conn({'earned_exam1': earned_by_exid['exam1'], 'earned_exam2': earned_by_exid['exam2'],
                    'earned_exam3': earned_by_exid['exam3']})

### Select first five rows from exam table 'earned_exam1', order by email alphabetically in ascending order:
query = '''
SELECT
*
FROM earned_exam1
ORDER BY email
LIMIT 5
'''

df = pd.read_sql_query(query, conn)
print("From table 'earned_exam1':")
display(df)

If the email specified is `'aaliyah.tayviona@gatech.edu'`, the exam_table is `'earned_exam1'`, and the max_score is `50.0`, then the demo included in the solution cell below should display the following output, as `(23.0 / 50.0 * 100.0) = 46.0`:

|    | Email                           | Percentage Earned  |
|---:|:--------------------------------|:------------|
|  0 | aaliyah.tayviona@gatech.edu     | 46.0    |



## Exercise 3 your code ##

In [ ]:
### Exercise 3 solution
def compute_percentage_earned(conn, email, exam_table, max_score):
    ### BEGIN SOLUTION
    query = '''

    '''
    ### END SOLUTION

    df = pd.read_sql_query(query, conn)
    return df


<!-- Test Cell Boilerplate -->
## Exercise 3 test ##
The cell below will test your solution for Exercise 3.

In [ ]:
### test_cell_ex3
from pandas.testing import assert_frame_equal

assert isinstance(compute_percentage_earned(conn, 'aaliyah.tayviona@gatech.edu', 'earned_exam1', 50.0), pd.DataFrame), 'Sorry, your result is not a dataframe!'

test_df0 = pd.DataFrame({'Email': 'aaliyah.tayviona@gatech.edu', 'Percentage Earned': 46.0}, index=[0])
assert_frame_equal(compute_percentage_earned(conn, 'aaliyah.tayviona@gatech.edu', 'earned_exam1', 50.0), test_df0)

test_df1 = pd.DataFrame({'Email': 'zuri.eilleen@gatech.edu', 'Percentage Earned': 75.0}, index=[0])
assert_frame_equal(compute_percentage_earned(conn, 'zuri.eilleen@gatech.edu', 'earned_exam3', 60.0), test_df1)

test_df2 = pd.DataFrame({'Email': 'sharon.syniya@gmail.com', 'Percentage Earned': 100.0}, index=[0])
assert_frame_equal(compute_percentage_earned(conn, 'sharon.syniya@gmail.com', 'earned_exam2', 50.0), test_df2)

test_df3 = pd.DataFrame({'Email': 'dennis.nolin@gmail.com', 'Percentage Earned': 8.0}, index=[0])
assert_frame_equal(compute_percentage_earned(conn, 'dennis.nolin@gmail.com', 'earned_exam2', 50.0), test_df3)

test_df4 = pd.DataFrame({'Email': 'joseph.sherburne@gmail.com', 'Percentage Earned': 45.833333}, index=[0])
assert_frame_equal(compute_percentage_earned(conn, 'joseph.sherburne@gmail.com', 'earned_exam3', 60.0), test_df4)

print('Passed! Please continue.')

# Exercise 4: `compute_matrix_product` #

Given two 2-D Numpy arrays, complete the function,
```python
def compute_matrix_product(arr1, arr2):
    ...
```
so that it replaces any values greater than `10` with `10` in either array. Then, if it is possible, compute the matrix product of the two arrays, and return the result. If they cannot be multiplied, your function should return the string `'Cannot be multiplied'`

**Inputs:**
- `arr1`: A Numpy 2-D array
- `arr2`: A Numpy 2-D array

**Your tasks:**
- Determine if the two arrays can be multiplied by comparing their shapes. If they cannot be multiplied, return `'Cannot be multiplied'`
- Replace any values greater than `10` in each array with the value `10`
- Find the matrix product of the two arrays, and return the result.

**Output:** A Numpy array.

**Hint:** Check out the documentation [here](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) for more information on matrix multiplication.
    

## Exercise 4 demo input ##

In [ ]:
### Define demo inputs ###

### Array 1:
demo_arr1_ex4 = np.array([[10, 11], [8, 6]])
print('Array 1:')
print(demo_arr1_ex4)
print()

### Array 2:
demo_arr2_ex4 = np.array([[4, 1], [2, 2]])
print('Array 2:')
print(demo_arr2_ex4)

The demo included in the solution cell below should display the following output:
```python
array([[60, 30],
       [44, 20]])
```

## Exercise 4 your code ##

In [ ]:
### Exercise 4 solution
def compute_matrix_product(arr1, arr2):
    ### BEGIN SOLUTION

    ### END SOLUTION

### demo function call ###
compute_matrix_product(demo_arr1_ex4, demo_arr2_ex4)

<!-- Test Cell Boilerplate -->
## Exercise 4 test ##
The cell below will test your solution for Exercise 4.

In [ ]:
### test_cell_ex4
from numpy.testing import assert_array_equal

assert isinstance(compute_matrix_product(demo_arr1_ex4, demo_arr2_ex4), np.ndarray), 'Sorry, your result is not an array!'

assert_array_equal(compute_matrix_product(demo_arr1_ex4, demo_arr2_ex4), np.array([[60, 30],[44, 20]]))

assert_array_equal(compute_matrix_product(np.array([[20, 20], [20, 20]]), np.array([[11, 11], [11, 11]])),
                   np.array([[200, 200],[200, 200]]))

assert_array_equal(compute_matrix_product(np.array([[1, 1], [1, 1]]), np.array([[0, 1], [1, 1]])),
                   np.array([[1, 2],[1, 2]]))

assert compute_matrix_product(np.array([[1, 1], [1, 1]]), np.array([[0, 1], [1, 1], [10, 12]])) == 'Cannot be multiplied', 'Sorry, these arrays cannot be multiplied!'

assert_array_equal(compute_matrix_product(np.array([[1, 1, 2], [1, 1, 4], [100, 3, 5]]), np.array([[0, 1], [1, 1], [10, 12]])),
                   np.array([[21, 22],[41, 42],[53, 63]]))

print('Passed! Congratulations!')